# Helper functions

In [ ]:
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV
import numpy as np
import scipy
import scipy.stats as sp
import matplotlib.pyplot as plt
from scipy.stats import binned_statistic
from sklearn.cluster import MiniBatchKMeans

In [ ]:
rng = np.random.default_rng(seed=42)

trials = 500
n_max = 10000
d_max = 5
m_max = 59
alpha_param_space = np.logspace(1, 2, 5)
gamma_param_space = np.logspace(-2, 2, 5)

def generate_data_randomness(trials, n_max, d_max, m_max):

  ny = rng.normal(size=(n_max,1, trials))
  z = rng.normal(0, 4, size = (n_max, d_max, trials))
  nx = rng.normal(size=(n_max, m_max+1, trials))

  return nx, ny, z


nx_big, ny_big, z_big = generate_data_randomness(trials = trials, n_max = n_max, d_max = d_max, m_max = m_max)
print(nx_big.shape)
print(ny_big.shape)
print(z_big.shape)
print(nx_big[0][0][0])

In [ ]:
# key: (d, beta, a, n)
params_x_stored = {
            (1, 0, 2, 10000): {'alpha': 17.78279410038923, 'gamma': 1.0},
            (5, 0, 2, 10000): {'alpha': 56.23413251903491, 'gamma': 0.1},
            (1, 0, 2, 3160) : {'alpha': 10.0, 'gamma': 1.0},
            (5, 0, 2, 3160) : {'alpha': 10.0, 'gamma': 10.0},
            }
params_y_stored = {
            (1, 0, 2, 10000): {'alpha': 17.78279410038923, 'gamma': 1.0},
            (5, 0, 2, 10000) : {'alpha': 56.23413251903491, 'gamma': 0.1},
            (1, 0.25, 2, 10000) : {'alpha': 31.622776601683793, 'gamma': 1.0},
            (5, 0.25, 2, 10000) : {'alpha': 10.0, 'gamma': 1.0},
            (1, 0.5, 2, 10000) : {'alpha': 10.0, 'gamma': 1.0},
            (5, 0.5, 2, 10000) : {'alpha': 31.622776601683793, 'gamma': 0.1},
            (1, 1.5, 2, 10000) : {'alpha': 10.0, 'gamma': 1.0},
            (5, 1.5, 2, 10000) : {'alpha': 31.622776601683793, 'gamma': 0.1},
            (1, 0, 2, 3160) : {'alpha': 10.0, 'gamma': 1.0},
            (5, 0, 2, 3160) : {'alpha': 10.0, 'gamma': 10.0},
            (1, 0.5, 2, 3160) :{'alpha': 10.0, 'gamma': 1.0},
            (5, 0.5, 2, 3160) :{'alpha': 10.0, 'gamma': 1.0},
            (1, 1.5, 2, 3160) :{'alpha': 10.0, 'gamma': 1.0},
            (5, 1.5, 2, 3160) : {'alpha': 10.0, 'gamma': 1.0}
            }

### GCM functions

In [ ]:
def f_a(s, a = 6.0):

  return np.exp(-s*s/2) * np.sin(a*s)


def generate_gcm_final(nx_big, ny_big, z_big, trial, a = 2, beta = 0, d = 1, n = 500):

  nx = np.copy(nx_big[:, 0:1, trial])
  ny = np.copy(ny_big[:, :, trial])
  z = np.copy(z_big[:, 0:d, trial])
  x = f_a(z[:, 0:1], a=a) + nx
  y = - 1* f_a(z[:, 0:1], a=a) + ny + beta*x

  x_max = np.max(x)
  y_max = np.max(y)

  true_res_x = nx / x_max
  true_res_y = (ny + beta*nx) / y_max
  true_res = true_res_x*true_res_y

  x = rescale(x)
  y = rescale(y)

  return x[:n], y[:n], z[:n], true_res[:n]


In [ ]:
def gcm(res, private = False, eps = 1.0, l1_sens = 0):

  # Input: res - residuals products
  #        private - whether the score is computed privately or not
  # Returns: t_stat - value of the GCM score T(x, y, z) for whether x \perp y | z
  #          p_val - p value of the test T(x, y, z)

  n = res.shape[0]
  if private:
    noise_std = l1_sens / eps
    res_noise = rng.laplace(0, scale = noise_std,  size = (n, 1))
    res = res + res_noise

  numerator = np.sum(res) / np.sqrt(n)
  denominator = np.std(res, dtype=np.float64)

  if denominator == 0:
    return 0, 1

  t_stat = numerator/denominator
  p_val = sp.norm.sf(abs(t_stat))*2

  return t_stat, p_val


def residuals(x, y, z, params_x = None, params_y = None):

  # Input: variables x, y, z.
  #        x and y have shape n x 1, z has shape n x m for arbitrary m

  # Returns: res - product of residuals, res_x - residuls of fitting x to z, res_y - residulas of fitting y to z


  # Compute residuals of fitting a linear regression model of x to z and fitting y to z
  model_x, model_y = None, None
  x_orig, y_orig, z_orig = np.copy(x), np.copy(y), np.copy(z)

  if x.shape[0] > 1000:
    x, y, z = subsample(x, y, z, 1000)
  if params_x == None:
    model_x = GridSearchCV(
      KernelRidge(kernel="rbf", gamma=1),
      param_grid={"alpha": alpha_param_space, "gamma": gamma_param_space},
      cv=5)
    model_y = GridSearchCV(
      KernelRidge(kernel="rbf", gamma=1),
      param_grid={"alpha": alpha_param_space, "gamma": gamma_param_space},
      cv=5)
  else:
    model_x = KernelRidge(kernel = "rbf", alpha = params_x["alpha"], gamma = params_x["gamma"])
    model_y = KernelRidge(kernel = "rbf", alpha = params_y["alpha"], gamma = params_y["gamma"])

  model_x.fit(z, x)
  model_y.fit(z, y)

  if params_x == None:
    params_x = model_x.best_params_
    params_y = model_y.best_params_

  pred_x = model_x.predict(z_orig)
  pred_y = model_y.predict(z_orig)
  res_x = x_orig - pred_x
  res_y = y_orig - pred_y
  res = np.multiply(res_x, res_y)

  return res, pred_x, pred_y, params_x, params_y


def subsample(x, y, z, m):
  n = x.shape[0]
  sampled_rows = np.random.choice(n, m, replace = False)
  return x[sampled_rows], y[sampled_rows], z[sampled_rows]


def subsample_crt(y, z, k):
  n = x.shape[0]
  sampled_rows = np.random.choice(n, k, replace = False)
  return y[sampled_rows], z[sampled_rows]

def rescale(x):

  max_x = max(np.max(np.abs(x)), [1])
  return x / max_x

def calc_l1_sens(n, lam):
  a = 2 + 2*np.sqrt(2)/np.sqrt(lam)
  b = 8*np.sqrt(2)/(lam**(3/2) * n) + 8/(lam*n)
  return np.sqrt(n*a*a*b*b + 2*(a**3)*b + a**4)

def calc_l1_sens_laplace(lam):
  a = 2 + 2*np.sqrt(2)/np.sqrt(lam)
  b = 8*np.sqrt(2)/(lam**(3/2)) + 8/lam

  return a*b + a*a

def calc_sens_crt(lam):
  a = 2 + 2*np.sqrt(2)/np.sqrt(lam)
  b = 8*np.sqrt(2)/(lam**(3/2)) + 8/lam

  return 2*a + b


def conf_inv(power, trials):

  count = power * trials
  ci_low = np.zeros_like(power)
  ci_high = np.zeros_like(power)

  for i in range(power.shape[0]):
    for j in range(power.shape[1]):

      ci = sp.binomtest(int(count[i][j]), trials).proportion_ci()
      ci_low[i][j] = ci[0]
      ci_high[i][j] = ci[1]


  return ci_low, ci_high

In [ ]:
def determine_params_vary_n(trial, params_x_list, params_y_list, a, beta, n, dims, d, j, k):

  if n > 1000:
    params_x = params_x_stored[(dims, 0, a, n)]
    params_y = params_y_stored[(dims, beta, a, n)]
  else:
    if  trial == 0:
      params_x = None
      params_y = None
    else:
      params_x = params_x_list[d][j][k]
      params_y = params_y_list[d][j][k]

  return params_x, params_y

def gcm_experiment(x, y, z, true_res, params_x, params_y, eps):

  n = x.shape[0]
  reject, reject_priv, reject_oracle = 0, 0, 0

  res, pred_x, pred_y, params_x, params_y  = residuals(x, y, z, params_x = params_x, params_y = params_y)

  t_stat, p_val = gcm(res, private = False)
  if p_val <= 0.05: reject = 1

  lam = 2*min(params_x["alpha"], params_y["alpha"])
  l1_sens = calc_l1_sens_laplace(lam)

  t_stat, p_val_priv = gcm(res, private = True, eps = eps, l1_sens = l1_sens)
  if p_val_priv <= 0.05: reject_priv = 1

  # t_stat, p_val_oracle = gcm(true_res, private = True, eps = eps, l1_sens = l1_sens)
  # if p_val_oracle <= 0.05: reject_oracle = 1

  return reject, reject_priv, reject_oracle, params_x, params_y

### CRT functions

In [ ]:
def generate_crt_final(nx_big, ny_big, z_big, trial, n, d, beta, a, m):

  nx = np.copy(nx_big[:, 0:m+1, trial])
  ny = np.copy(ny_big[:, :, trial])
  z = np.copy(z_big[:, 0:d, trial])

  max_nx = np.max(np.abs(nx), axis = 0, keepdims = True)

  res_x = nx / max_nx

  faz = f_a(z[:, 0:1], a)

  x_column = faz + nx[:, 0:1]

  y = -1 * faz + ny + beta*x_column

  y_max = np.max(y)

  y = rescale(y)

  true_res_y = (beta*nx[:, 0:1] + ny) / y_max

  return res_x[:n, ], y[:n, ], z[:n, 0:d] , true_res_y[:n]



def residuals_crt(y, z, params_y = None):

  # Input: variables x, y, z.
  #        x and y have shape n x 1, z has shape n x m for arbitrary m

  # Returns: res - product of residuals, res_x - residuls of fitting x to z, res_y - residulas of fitting y to z


  # Compute residuals of fitting a linear regression model of x to z and fitting y to z
  model_y = None
  y_orig, z_orig = np.copy(y), np.copy(z)
  subsampled = True if y.shape[0] > 1000 else False
  if subsampled:
    y, z = y[:1000], z[:1000]
  if params_y == None:
    model_y = GridSearchCV(
      KernelRidge(kernel="rbf", gamma=1),
      param_grid={"alpha": alpha_param_space, "gamma": gamma_param_space},
      cv=5)
  else:
    model_y = KernelRidge(kernel = "rbf", alpha = params_y["alpha"], gamma = params_y["gamma"])

  model_y.fit(z, y)

  if params_y == None:
    params_y = model_y.best_params_

  pred_y = model_y.predict(z_orig)
  res_y = y_orig - pred_y

  return res_y, pred_y, params_y


def calc_p_val(res_x, res_y, true_res_y, eps_final = 10, l1_sens = 10):


    m = res_x.shape[1]

    t = np.zeros(m)
    t_true = np.zeros(m)

    gamma = 4*l1_sens*np.log2(m)/eps_final


    for j in range(m):
      res_x_column = res_x[:, j]
      res = res_x_column * np.squeeze(res_y)
      t[j] = np.sum(res)
      t_true[j] = np.sum(res_x_column * np.squeeze(true_res_y))

    dist_t0 = np.abs(t - t[0])
    c_gamma = np.sum(dist_t0 < gamma)


    # non_private
    threshold = t[0]
    count = np.sum(t > threshold)
    p_val = (count+1)/m

    reject = 1 if p_val <= 0.05 else 0


    # private
    sorted_t = sorted(t, reverse = True)
    score = np.zeros(m)
    for i in range(m):
      score[i] = -1*(np.abs(sorted_t[i] - t[0]))/(2*l1_sens)


    # oracle
    sorted_t_true = sorted(t_true, reverse = True)
    score_true = np.zeros(m)
    for i in range(m):
      score_true[i] = -1*(np.abs(sorted_t_true[i] - t_true[0]))/(2*l1_sens)


    noise = np.random.exponential(scale = 2.0/eps_final, size = (m,))
    noisy_scores = score + noise
    noisy_scores_true = score_true + noise


    rank = np.argmax(noisy_scores)
    p_val_priv = (rank+1)/m
    reject_priv = 1 if p_val_priv <= 0.05 else 0

    rank_true = np.argmax(noisy_scores)
    p_val_true = (rank+1)/m
    reject_true = 1 if p_val_true <= 0.05 else 0

    return reject, reject_priv, c_gamma, p_val, p_val_priv

### Kendall functions

The following code was adapted from the github repository: https://github.com/sunblaze-ucb/Priv-PC-Differentially-Private-Causal-Graph-Discovery

In [ ]:
# Kendall Conditional Test

import numpy as np
from collections import namedtuple
import warnings
from numpy import ma
from scipy.stats import mstats_basic
from scipy.stats._stats import _kendall_dis
import scipy.special as special
from scipy.stats import norm


KendalltauResult = namedtuple('KendalltauResult', ('correlation', 'pvalue'))

def _contains_nan(a, nan_policy='propagate'):
    policies = ['propagate', 'raise', 'omit']
    if nan_policy not in policies:
        raise ValueError("nan_policy must be one of {%s}" %
                         ', '.join("'%s'" % s for s in policies))
    try:
        # Calling np.sum to avoid creating a huge array into memory
        # e.g. np.isnan(a).any()
        with np.errstate(invalid='ignore'):
            contains_nan = np.isnan(np.sum(a))
    except TypeError:
        # If the check cannot be properly performed we fallback to omitting
        # nan values and raising a warning. This can happen when attempting to
        # sum things that are not numbers (e.g. as in the function `mode`).
        contains_nan = False
        nan_policy = 'omit'
        warnings.warn("The input array could not be properly checked for nan "
                      "values. nan values will be ignored.", RuntimeWarning)

    if contains_nan and nan_policy == 'raise':
        raise ValueError("The input contains nan values")

    return (contains_nan, nan_policy)

def kendalltaua(x, y, initial_lexsort=None, nan_policy='propagate'):

    x = np.asarray(x).ravel()
    y = np.asarray(y).ravel()

    if x.size != y.size:
        raise ValueError("All inputs to `kendalltau` must be of the same size, "
                         "found x-size %s and y-size %s" % (x.size, y.size))
    elif not x.size or not y.size:
        print('here1 is wrong!')
        return KendalltauResult(np.nan, np.nan)  # Return NaN if arrays are empty

    # check both x and y
    cnx, npx = _contains_nan(x, nan_policy)
    cny, npy = _contains_nan(y, nan_policy)
    contains_nan = cnx or cny
    if npx == 'omit' or npy == 'omit':
        nan_policy = 'omit'

    if contains_nan and nan_policy == 'propagate':
        return KendalltauResult(np.nan, np.nan)

    elif contains_nan and nan_policy == 'omit':
        x = ma.masked_invalid(x)
        y = ma.masked_invalid(y)
        return mstats_basic.kendalltau(x, y)

    if initial_lexsort is not None:  # deprecate to drop!
        warnings.warn('"initial_lexsort" is gone!')

    def count_rank_tie(ranks):
        cnt = np.bincount(ranks).astype('int64', copy=False)
        cnt = cnt[cnt > 1]
        return ((cnt * (cnt - 1) // 2).sum(),
            (cnt * (cnt - 1.) * (cnt - 2)).sum(),
            (cnt * (cnt - 1.) * (2*cnt + 5)).sum(), cnt)

    size = x.size
    perm = np.argsort(y)  # sort on y and convert y to dense ranks
    x, y = x[perm], y[perm]
    y = np.r_[True, y[1:] != y[:-1]].cumsum(dtype=np.intp)

    # stable sort on x and convert x to dense ranks
    perm = np.argsort(x, kind='mergesort')
    x, y = x[perm], y[perm]
    x = np.r_[True, x[1:] != x[:-1]].cumsum(dtype=np.intp)

    dis = _kendall_dis(x, y)  # discordant pairs

    obs = np.r_[True, (x[1:] != x[:-1]) | (y[1:] != y[:-1]), True]
    cnt = np.diff(np.where(obs)[0]).astype('int64', copy=False)

    ntie = (cnt * (cnt - 1) // 2).sum()  # joint ties
    xtie, x0, x1, cntx = count_rank_tie(x)     # ties in x, stats
    ytie, y0, y1, cnty = count_rank_tie(y)     # ties in y, stats

    tot = (size * (size - 1)) // 2

    if xtie == tot or ytie == tot:
        return (KendalltauResult(np.nan, np.nan), cntx, cnty)

    # Note that tot = con + dis + (xtie - ntie) + (ytie - ntie) + ntie
    #               = con + dis + xtie + ytie - ntie
    con_minus_dis = tot - xtie - ytie + ntie - 2 * dis
    # tau = con_minus_dis / np.sqrt(tot - xtie) / np.sqrt(tot - ytie)
    tau = con_minus_dis / tot
    # Limit range to fix computational errors
    tau = min(1., max(-1., tau))

    # con_minus_dis is approx normally distributed with this variance [3]_
    var = (size * (size - 1) * (2.*size + 5) - x1 - y1) / 18. + (
        xtie * ytie) / (2. * size * (size - 1)) + x0 * y0 / (9. *
        size * (size - 1) * (size - 2))
    pvalue = special.erfc(np.abs(con_minus_dis) / np.sqrt(var) / np.sqrt(2))

    # Limit range to fix computational errors
    return KendalltauResult(min(1., max(-1., tau)), pvalue), cntx, cnty



def discondKendall(data_matrix, x, y, k, eps):
    s_size = len(k)
    k = k.copy()
    row_size = data_matrix.shape[0]
    if s_size == 0:
        (tau, pval), _, _ = kendalltaua(data_matrix[:,x], data_matrix[:,y])
        tau = tau * np.sqrt(9.0 * row_size * (row_size - 1) / (4*row_size+10))
        pval = norm.sf(np.abs(tau))
        return tau, pval
    z = []
    for z_index in range(s_size):
        z.append(k.pop())
        pass


    dm_unique = np.unique(data_matrix[:, z], axis=0)

    sumwk = 0
    sumweight = 0
    tau = 0
    pval = 0
    for split_k in dm_unique:
        index = np.ones((row_size),dtype=bool)
        for i in range(s_size):
            index = ((data_matrix[..., z[i]] == split_k[i]) & index)

        new_dm = data_matrix[index, :]
        nk = new_dm.shape[0]
        if nk <= 2:
            continue
        (condtau, condpval), cntx, cnty = kendalltaua(new_dm[:, x], new_dm[:, y])
        if np.isnan(condpval):
            continue
        sigma0_sq = (4.0 * nk + 10) / (9.0 * nk * (nk-1.0))
        tau += condtau / sigma0_sq
        sumwk += 1.0 / sigma0_sq

    tau /= np.sqrt(sumwk)

    #iden lines
    noise = 0
    if eps > 0:
      noise = np.random.laplace(0, scale = 4.5/(eps*np.sqrt(row_size-1)))


    # print(tau, noise)
    tau = tau + noise
    #S, _ = sp.norm.quad(lambda x: np.exp(-x**2/2) / np.sqrt(2*np.pi), 0, 6 / np.sqrt(row_size))
    #noise = np.random.laplace(0, scale=S/eps)

    pval = 2*norm.sf(np.abs(tau))


    return tau, pval

def binned(z, bins):
  bin_means, bin_edges, bin_number = binned_statistic(np.squeeze(z), np.squeeze(z), bins=bins)
  binned_z = np.zeros_like(z)

  for i in range(len(z)):
    binned_z[i] = bin_means[bin_number[i]-1]

  return binned_z



def cluster(z, n_clusters = 10):

  model = MiniBatchKMeans(n_clusters = n_clusters, batch_size = 500, n_init = 3)
  model.fit(z)
  centers, labels = model.cluster_centers_, model.labels_
  clustered_z = np.zeros_like(z)
  for i in range(z.shape[0]):
    clustered_z[i] = centers[labels[i]]
  return clustered_z



def kendall_tau(x, y, z, n_clusters = 10, eps = 1, alpha = 0.05):

  n = z.shape[0]
  z_binned = cluster(z, n_clusters = n_clusters)
  data_matrix = np.concatenate((x,y,z_binned), axis=1)
  z_indices = set()
  z_indices.add(2)
  tau_priv, p_val_priv = discondKendall(data_matrix, 0, 1, z_indices, eps = eps)
  reject = 1 if p_val_priv < alpha else 0

  return reject

### TOT

The following code was adapted from the github repository: https://github.com/diff-priv-ht/test-of-tests

In [ ]:
def rtulap(m, b):

  G1 = np.random.geometric(1 - b)
  G2 = np.random.geometric(1 - b)
  U = np.random.uniform(low=-0.5, high=0.5)

  noise = G1 - G2 + U + m

  return noise


# CDF of Tulap(m,b)
# Definition 4.1 in Awan & Slavkovic (2018)
def ptulap(m, b, x):

  ret = 0
  if x <= np.round(m):
    ret = (b**(- round(x-m))/ (1 + b)) * (b + (x - m - round(x - m) + 1/2) * (1 - b))
  else:
    ret = 1 - ((b**round(x - m)) / (1 + b)) * (b + (round(x - m) - (x - m) + 1/2) * (1 - b))

  return ret

# Compute p-value given test statistic, Z
# Algorithm 1 in Awan & Slavkovic (2018)
def dp_binom_pval(n, alpha0, eps, noise):

  f = lambda t : ptulap(0, np.exp(-eps), t-noise)
  F_underbar = np.array([f(t) for t in range(n+1)])


  f = lambda t : scipy.special.comb(n, t) * alpha0**t * (1 - alpha0)**(n - t)
  B_underbar = np.array([f(t) for t in range(n+1)])

  return(sum(F_underbar * B_underbar))



def test_of_tests(x,y, z, true_res, eps, m, alpha0 = 0.05, params_x = None, params_y = None):

  # Partition the data into random subsamples
  n = x.shape[0]
  k = int(n / m)
  sub_tests = np.zeros(m)

  for i in range(m):
    # Compute p-value in each subsample
    x_sample, y_sample, z_sample, true_res_sample = x[i*k:(i+1)*k], y[i*k:(i+1)*k], z[i*k:(i+1)*k], true_res[i*k:(i+1)*k]
    reject, reject_priv, reject_oracle, params_x, params_y = gcm_experiment(x_sample, y_sample, z_sample, true_res_sample, params_x = params_x, params_y = params_y, eps = eps)
    sub_tests[i] = reject

  # Run the Awan & Slavkovic (2018) private binomial test
  noise = rtulap(m = sum(sub_tests == 1), b = np.exp(-eps))
  p_val = dp_binom_pval(n = m, alpha0 = alpha0, eps = eps, noise = noise)
  reject = 1 if p_val <= 0.05 else 0
  return(reject, params_x, params_y)


# CRT

### **Vary m**

In [ ]:
trials = 500
alpha = 0.05
n = 1000
a = 2.0
var = 1
d_list = [1, 5]


beta_list = [0, 0.5, 1.5]
m_list = [19, 29, 39, 49, 59]

power = np.zeros((len(d_list), len(beta_list), len(m_list)))
power_priv = np.zeros_like(power)
c_gamma_list = np.zeros_like(power)

params_y_list = [[0]*len(m_list)]*len(d_list)


eps_final = 2


for i in range(1,trials+1):

  for j in range(len(beta_list)):

    beta = beta_list[j]

    for d in range(len(d_list)):

      dims = d_list[d]

      res_x, y, z, true_res_y = generate_crt_final(nx_big, ny_big, z_big, trial = i-1, n=n, d=dims, a=a, beta=beta, m = m_list[-1])

      if i == 1:
        res_y, pred_y, params_y = residuals_crt(y, z)
        params_y_list[d][j] = params_y
      else:
        res_y = residuals_crt(y, z, params_y = params_y_list[d][j])[0]

      l1_sens = calc_sens_crt(lam = 2*params_y_list[d][j]['alpha'])

      for k in range(len(m_list)):

        m = m_list[k]

        res_x_slice = np.copy(res_x[:, :m+1])

        reject, reject_priv, c_gamma, p_val, p_val_priv = calc_p_val(res_x_slice, res_y, true_res_y, eps_final = eps_final, l1_sens = l1_sens)

        power[d][j][k] += reject
        power_priv[d][j][k] += reject_priv

        c_gamma_list[d][j][k] += c_gamma

  if i % 100 == 0:
    print("Done with {} trials".format(i))
    print(power/i)
    print(power_priv/i)
    print("\n")



print(np.array2string(power/trials, separator = ","))
print(np.array2string(power_priv/trials, separator = ","))
print(np.array2string(c_gamma_list/trials, separator = ","))

### **Vary $n$**

In [ ]:
trials = 500
alpha = 0.05
m=19
a = 2.0
var = 1
d_list = [1, 5]


beta_list = [0, 0.5, 1.5]
n_list = [100, 310, 1000, 3160, 10000]

power = np.zeros((len(d_list), len(beta_list), len(n_list)))
power_priv = np.zeros_like(power)
c_gamma_list = np.zeros_like(power)

params_y_list = power.tolist()


eps_final = 2


for i in range(1,trials+1):

  for j in range(len(beta_list)):

    beta = beta_list[j]

    for k in range(len(n_list)):

      n = n_list[k]

      for d in range(len(d_list)):

        dims = d_list[d]

        res_x, y, z, true_res_y = generate_crt_final(nx_big, ny_big, z_big, trial = i-1, n=n, d=dims, a=a, beta=beta, m = m)

        params_y = None if i == 1 else params_y_list[d][j][k]
        res_y, pred_y, params_y = residuals_crt(y, z, params_y = params_y)
        params_y_list[d][j][k] = params_y


        l1_sens = calc_sens_crt(lam = 2*params_y['alpha'])


        reject, reject_priv, c_gamma, p_val, p_val_priv = calc_p_val(res_x, res_y, true_res_y, eps_final = eps_final, l1_sens = l1_sens)

        power[d][j][k] += reject
        power_priv[d][j][k] += reject_priv

        c_gamma_list[d][j][k] += c_gamma

  if i == 1: print("Done with 1 trial")
  if i % 100 == 0:
    print("Done with {} trials".format(i))
    print(power/i)
    print(power_priv/i)
    print("\n")

print(np.array2string(power/trials, separator = ","))
print(np.array2string(power_priv/trials, separator = ","))
print(np.array2string(c_gamma_list/trials, separator = ","))

### **Vary $a$**

In [ ]:
trials = 500
alpha = 0.05
n = 1000
m=19
a = 2.0
var = 1
d_list = [1, 5]


beta_list = [0, 0.5, 1.5]
a_list = [1, 2, 4, 8, 16, 32]


power = np.zeros((len(d_list), len(beta_list), len(a_list)))
power_priv = np.zeros_like(power)
c_gamma_list = np.zeros_like(power)

params_y_list = power.tolist()


eps_final = 2


for i in range(1,trials+1):

  for j in range(len(beta_list)):

    beta = beta_list[j]

    for k in range(len(a_list)):

      a = a_list[k]

      for d in range(len(d_list)):

        dims = d_list[d]

        res_x, y, z, true_res_y = generate_crt_final(nx_big, ny_big, z_big, trial = trials[i-1], n=n, d=dims, a=a, beta=beta, m = m)

        if i == 1:
          res_y, pred_y, params_y = residuals_crt(y, z)
          params_y_list[d][j][k] = params_y
        else:
          res_y = residuals_crt(y, z, params_y = params_y_list[d][j][k])[0]

        l1_sens = calc_sens_crt(lam = 2*params_y_list[d][j][k]['alpha'])


        reject, reject_priv, c_gamma, p_val, p_val_priv = calc_p_val(res_x, res_y, eps_final = eps_final, l1_sens = l1_sens)

        if p_val <= alpha: power[d][j][k] += 1
        if p_val_priv <= alpha: power_priv[d][j][k] += 1

        c_gamma_list[d][j][k] += c_gamma

  if i % 100 == 0:
    print("Done with {} trials".format(i))
    print(power/i)
    print(power_priv/i)
    print("\n")

power = power / trials
power_priv = power_priv / trials
c_gamma_list = c_gamma_list/trials

print(np.array2string(power, separator = ","))
print(np.array2string(power_priv, separator = ","))
print(np.array2string(c_gamma_list, separator = ","))

### **Vary $\varepsilon$**

In [ ]:
trials = 500
alpha = 0.05
n = 10**4
m=19
a = 2.0
var = 1
d_list = [1, 5]


beta_list = [0, 0.5, 1.5]
eps_list = [1/8, 1/4, 1/2, 1, 2, 4, 8]


power_gcm = np.zeros((len(d_list), len(beta_list), len(eps_list)))
power_crt = np.zeros_like(power_gcm)

params_y_list = [[0]*len(beta_list)]*len(d_list)
params_x_list = [[0]*len(beta_list)]*len(d_list)


for i in range(1,trials+1):

  for j in range(len(beta_list)):

    beta = beta_list[j]

    for d in range(len(d_list)):

      dims = d_list[d]

      res_x, y, z, true_res_y = generate_crt_final(nx_big, ny_big, z_big, trial = i-1, n=n, d=dims, a=a, beta=beta, m = m)
      x, y, z, true_res = generate_gcm_final(nx_big, ny_big, z_big, trial = i-1, n=n, d=dims, a=a, beta=beta)



      params_x = None if i == 1 else params_x_list[d][j]
      params_y = None if i == 1 else params_y_list[d][j]

      res_y, pred_y, params_y = residuals_crt(y, z, params_y)
      res_x_gcm, pred_x, params_x = residuals_crt(x, z, params_x)
      res = res_x_gcm * res_y
      params_x_list[d][j], params_y_list[d][j] = params_x, params_y


      l1_sens_crt = calc_sens_crt(lam = 2*params_y_list[d][j]['alpha'])
      lam = 2*min(params_x_list[d][j]["alpha"], params_y_list[d][j]["alpha"])
      l1_sens_gcm = calc_l1_sens_laplace(lam=lam)

      for k in range(len(eps_list)):

        eps = eps_list[k]

        reject, reject_priv, c_gamma, p_val, p_val_priv = calc_p_val(res_x, res_y, true_res_y, eps_final = eps, l1_sens = l1_sens_crt)

        power_crt[d][j][k] += reject_priv

        t_stat, p_val_gcm = gcm(res, private=True, eps = eps, l1_sens = l1_sens_gcm)

        if p_val_gcm <= 0.05: power_gcm[d][j][k] += 1


  if i % 100 == 0:
    print("Done with {} trials".format(i))
    print(power_gcm/i)
    print(power_crt/i)
    print("\n")


print(np.array2string(power_gcm/trials, separator = ","))
print(np.array2string(power_crt/trials, separator = ","))

### **Vary $\beta$**

In [ ]:
beta_list = [0, 0.25, 0.5, 0.75, 1, 1.25, 1.5]

trials = 500
alpha = 0.05
n = 1000
m=19
a = 2.0
var = 1
d_list = [1, 5]

a=2

power = np.zeros((len(d_list), len(beta_list)))
power_priv = np.zeros_like(power)
power_oracle = np.zeros_like(power)

params_y_list = power.tolist()

eps_final = 2

for i in range(1,trials+1):

  for j in range(len(beta_list)):

    beta = beta_list[j]

    for d in range(len(d_list)):

      dims = d_list[d]

      res_x, y, z, true_res_y = generate_crt_final(nx_big, ny_big, z_big, trial = i-1, n=n, d=dims, a=a, beta=beta, m=m)

      params_y = None if i == 1 else params_y_list[d][j]
      res_y, pred_y, params_y = residuals_crt(y, z, params_y = params_y)
      params_y_list[d][j] = params_y


      l1_sens = calc_sens_crt(lam = 2*params_y_list[d][j]['alpha'])


      reject, reject_priv, c_gamma, p_val, p_val_priv = calc_p_val(res_x, res_y, true_res_y, eps_final = eps_final, l1_sens = l1_sens)

      power[d][j] += reject
      power_priv[d][j] += reject_priv
      power_oracle[d][j] += reject_oracle


  if i % 100 == 0:
    print("Done with {} trials".format(i))
    print(power/i)
    print(power_priv/i)
    print("\n")

print(np.array2string(power/trials, separator = ","))
print(np.array2string(power_priv/trials, separator = ","))
print(np.array2string(power_oracle/trials, separator = ","))

In [ ]:
beta_list = [0, 0.5, 1.5]

trials = 500
alpha = 0.05
n = 1000
m=19
a = 2.0
var = 1
d_list = [1, 5]


power = np.zeros((len(d_list), len(beta_list)))
power_priv = np.zeros_like(power)
power_oracle = np.zeros_like(power)
pvals = np.zeros((len(d_list), len(beta_list), trials))
pvals_priv = np.zeros_like(pvals)

params_y_list = power.tolist()

eps_final = 2

for i in range(1,trials+1):

  for j in range(len(beta_list)):

    beta = beta_list[j]

    for d in range(len(d_list)):

      dims = d_list[d]

      res_x, y, z, true_res_y = generate_crt_final(nx_big, ny_big, z_big, trial = i-1, n=n, d=dims, a=a, beta=beta, m=m)

      params_y = None if i == 1 else params_y_list[d][j]
      res_y, pred_y, params_y = residuals_crt(y, z, params_y = params_y)
      params_y_list[d][j] = params_y


      l1_sens = calc_sens_crt(lam = 2*params_y_list[d][j]['alpha'])


      reject, reject_priv, reject_oracle, c_gamma, pval, pval_priv = calc_p_val(res_x, res_y, true_res_y, eps_final = eps_final, l1_sens = l1_sens)

      power[d][j] += reject
      power_priv[d][j] += reject_priv
      power_oracle[d][j] += reject_oracle
      pvals[d][j][i-1] = pval
      pvals_priv[d][j][i-1] = pval_priv


  if i % 100 == 0:
    print("Done with {} trials".format(i))
    print(power/i)
    print(power_priv/i)
    print("\n")

print(np.array2string(power/trials, separator = ","))
print(np.array2string(power_priv/trials, separator = ","))
print(np.array2string(power_oracle/trials, separator = ","))
print(np.array2string(pvals, separator = ","))
print(np.array2string(pvals_priv, separator = ","))

# GCM

### **Vary $a$**

In [ ]:
trials = 500
alpha = 0.05
n = 10**4

eps = 2
var = 1
m = 19

a_list = [1, 2, 4, 8, 16, 32]
d_list = [1,5]
beta = 0

power = np.zeros((len(d_list), len(a_list)))
power_gcm = np.zeros_like(power)
power_crt = np.zeros_like(power)
power_kendall = np.zeros_like(power)
power_tot = np.zeros_like(power)
power_gcm_nonpriv = np.zeros_like(power)
params_x_list = power.tolist()
params_y_list = power.tolist()
params_x_list_tot = power.tolist()
params_y_list_tot = power.tolist()

for i in range(1, trials+1):

  for j in range(len(a_list)):

    a = a_list[j]


    for d in range(len(d_list)):

      dims = d_list[d]
      res_x, y, z, true_res_y = generate_crt_final(nx_big, ny_big, z_big, trial = i-1, n=n, d=dims, a=a, beta=beta, m = m)
      x, y, z, true_res = generate_gcm_final(nx_big, ny_big, z_big, trial = i-1, n=n, d=dims, a=a, beta=beta)

      # Kendall
      power_kendall[d][j] += kendall_tau(x, y, z, n_clusters = 100, eps = eps)

      # CRT and GCM
      params_x = None if i == 1 else params_x_list[d][j]
      params_y = None if i == 1 else params_y_list[d][j]

      res_x_fit, pred_x, params_x = residuals_crt(x, z, params_x)
      res_y_fit, pred_y, params_y = residuals_crt(y, z, params_y)

      params_x_list[d][j], params_y_list[d][j] = params_x, params_y

      l1_sens_crt = calc_sens_crt(lam = 2*params_y_list[d][j]['alpha'])
      lam = 2*min(params_x_list[d][j]["alpha"], params_y_list[d][j]["alpha"])
      l1_sens_gcm = calc_l1_sens_laplace(lam=lam)

      reject, reject_priv, c_gamma, p_val, p_val_priv = calc_p_val(res_x, res_y_fit, true_res_y, eps_final = eps, l1_sens = l1_sens_crt)

      power_crt[d][j] += reject_priv

      t_stat, p_val_gcm = gcm(res_x_fit*res_y_fit, private=True, eps = eps, l1_sens = l1_sens_gcm)

      if p_val_gcm <= 0.05: power_gcm[d][j] += 1

      t_stat, p_val_gcm = gcm(res_x_fit*res_y_fit, private=False)

      if p_val_gcm <= 0.05: power_gcm_nonpriv[d][j] += 1



      #TOT
      params_x = None if i == 1 else params_x_list_tot[d][j]
      params_y = None if i == 1 else params_y_list_tot[d][j]

      reject_tot, params_x, params_y = test_of_tests(x,y, z, true_res, eps=eps, m=10, params_x = params_x, params_y = params_y)
      power_tot[d][j] += reject_tot
      params_x_list_tot[d][j], params_y_list_tot[d][j] = params_x, params_y



  if i % 100 == 0:
    print("Done with {} trials".format(i))
    print(np.array2string(power_gcm_nonpriv/i, separator = ","))
    print(np.array2string(power_gcm/i, separator = ","))
    print(np.array2string(power_crt/i, separator = ","))
    print(np.array2string(power_kendall/i, separator = ","))
    print(np.array2string(power_tot/i, separator = ","))
    print("\n")

### **Vary $n$**

In [ ]:
trials = 500
alpha = 0.05

a = 2

eps = 7


beta_list = [0, 0.5, 1.5]
n_list = [100, 310, 1000, 3160, 10000]
d_list = [1, 5]
var = 1
power = np.zeros((len(d_list), len(beta_list), len(n_list)))
power_priv = np.zeros_like(power)
power_oracle = np.zeros_like(power)

params_x_list = power.tolist()
params_y_list = power.tolist()


for i in range(1, trials+1):

  for j in range(len(beta_list)):

    beta = beta_list[j]

    for k in range(len(n_list)):

      n = n_list[k]

      for d in range(len(d_list)):

        dims = d_list[d]
        x, y, z, true_res = generate_gcm_final(nx_big, ny_big, z_big, trial = i-1, n=n, d=dims, a=a, beta=beta)

        if i == 1:
          params_x, params_y = None, None
        else:
          params_x, params_y = params_x_list[d][j][k], params_y_list[d][j][k]

        reject, reject_priv, reject_oracle, params_x, params_y = gcm_experiment(x, y, z, true_res, params_x, params_y, eps = eps)
        params_x_list[d][j][k], params_y_list[d][j][k] = params_x, params_y

        power[d][j][k] += reject
        power_priv[d][j][k] += reject_priv
        power_oracle[d][j][k] += reject_oracle

  if i % 100 == 0:
    print("Done with {} trials".format(i))
    print(power/i)
    print(power_priv/i)
    print(power_oracle/i)
    print("\n")


print(np.array2string(power/trials, separator = ","))
print(np.array2string(power_priv/trials, separator = ","))
print(np.array2string(power_oracle/trials, separator = ","))

### Vary $\beta$

In [ ]:
trials = 500
alpha = 0.05

a = 2.0

eps = 7
n = 10**4

beta_list = [0, 0.25, 0.5, 0.75, 1, 1.25, 1.5]
d_list = [1, 5]

var = 1
power = np.zeros((len(d_list), len(beta_list)))
power_priv = np.zeros_like(power)
power_oracle = np.zeros_like(power)

params_x_list = power.tolist()
params_y_list = power.tolist()


for i in range(1, trials+1):


  for j in range(len(beta_list)):

    beta = beta_list[j]

    for d in range(len(d_list)):

      dims = d_list[d]
      x, y, z, true_res = generate_gcm_final(nx_big, ny_big, z_big, trial = i-1, n=n, d=dims, a=a, beta=beta)


      # GCM
      if i == 1:
        params_x, params_y = None, None
      else:
        params_x, params_y = params_x_list[d][j], params_y_list[d][j]

      reject, reject_priv, reject_oracle, params_x, params_y = gcm_experiment(x, y, z, true_res, params_x = params_x, params_y = params_y, eps = eps)
      params_x_list[d][j], params_y_list[d][j] = params_x, params_y

      power[d][j] += reject
      power_priv[d][j] += reject_priv
      power_oracle[d][j] += reject_oracle


  if i % 100 == 0:
    print("Done with {} trials".format(i))
    print(power/i)
    print(power_priv/i)
    print("\n")


power = power / trials
power_priv = power_priv / trials
power_oracle = power_oracle / trials


print(params_x_list, params_y_list)

print(np.array2string(power, separator = ","))
print(np.array2string(power_priv, separator = ","))
print(np.array2string(power_oracle, separator = ","))

# Concrete Strength Dataset

Requires filepath to locally stored Concerete Compressive Stregth Dataset .xls file from UCI Machine Learning Repository ([link](https://archive.ics.uci.edu/dataset/165/concrete+compressive+strength) to dataset)

In [ ]:
import pandas
fp = "Concrete_Data.xls"

data = pandas.read_excel(fp)
data = data.to_numpy()
print(data.shape)

In [ ]:
total_cols = data.shape[1]
trials = 500
eps = 7

true_res = None
data_size = [1000, 310, 100, 31, 10]

power = np.zeros((total_cols-1, (len(data_size))))
power_priv = np.zeros_like(power)

params_x, params_y = {'alpha': 10.0, 'gamma': 0.01}, {'alpha': 10.0, 'gamma': 0.01}

for column in range(total_cols-1):

  y = data[:, total_cols-1:total_cols]
  x = data[:, column:column+1]
  z1 = data[:, 0:column]
  z2 = data[:, column+1:total_cols-1]
  z = np.concatenate((z1, z2), axis = 1)
  x = rescale(x)
  y = rescale(y)

  for _ in range(trials):

    for i in range(len(data_size)):

      x_sub, y_sub, z_sub = subsample(x, y, z, data_size[i])
      reject, reject_priv, reject_oracle, params_x, params_y = gcm_experiment(x_sub, y_sub, z_sub, true_res, params_x, params_y, eps = eps)
      power[column][i] += reject
      power_priv[column][i] += reject_priv


print(np.array2string(power/trials, separator = ","))
print(np.array2string(power_priv/trials, separator = ","))